In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import requests
import json

random_state = 42

In [2]:
raw_data = pd.merge(pd.read_csv('../train_transaction.csv'), pd.read_csv('../train_identity.csv'),
                    left_on='TransactionID', right_on='TransactionID', how='left')


Подготовим данные для запроса, по которому выполняется обучение модели.

In [3]:
df_req_train = raw_data.sample(1000, random_state=random_state)
df_req_train.reset_index(drop=True, inplace=True)

Из тестового датасета возьмем 50 мошеннических и 50 немошеннических транзакций для тестирования запросов, по которым выплняется предсказание.

In [4]:
X_train, X_test = train_test_split(raw_data, test_size=0.2, random_state=random_state)

X_test.reset_index(drop=True, inplace=True)
y_test = X_test['isFraud']

del raw_data
del X_train

In [5]:
df_req_fraud = X_test[y_test==1].sample(50, random_state=random_state)
df_req_fraud.reset_index(drop=True, inplace=True)

df_req_not_fraud = X_test[y_test==0].sample(50, random_state=random_state)
df_req_not_fraud.reset_index(drop=True, inplace=True)

In [6]:
del X_test

### Запрос для одного объекта

Для каждой транзакции выполним отдельный запрос

In [22]:
%%time
url = 'http://127.0.0.1:8000/predict_one_transaction'
api_single_res = []
for i in range(df_req_fraud.shape[0]):
    response = requests.post(url, data=json.dumps(df_req_fraud.loc[i].to_dict()), headers={"Content-Type": "application/json"})
    api_single_res.append(response.json())

CPU times: total: 234 ms
Wall time: 3.83 s


Результаты запросов: 0 - транзакция не мошенническая, 1 - транзакция мошенническая

In [23]:
sum(api_single_res)

21.0

In [24]:
print(api_single_res)

[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]


### Запрос для списка транзакций

In [37]:
url_items = 'http://127.0.0.1:8000/predict_transactions'
lst = []
for i in range(df_req_fraud.shape[0]):
    lst.append(df_req_fraud.loc[i].to_dict())
response_items = requests.post(url_items, data=json.dumps(lst), headers={"Content-Type": "application/json"})
print(response_items.status_code)

200


In [38]:
sum(response_items.json())

13.0

In [39]:
print(response_items.json())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]


### Запрос для обучения модели

In [36]:
url_train = 'http://127.0.0.1:8000/train_full_data'
response_train = requests.post(url_train, data=json.dumps(df_req_train.to_dict(orient='list')), headers={"Content-Type": "application/json"})
response_train.json()

True

### Добавить транзакцию в БД

In [40]:
url_add_trn = 'http://127.0.0.1:8000/add_transaction'
for i in range(0, 1000, 100):
    response = requests.post(url_add_trn, data=json.dumps(df_req_train.loc[i].to_dict()), headers={"Content-Type": "application/json"})
    print(response.json())

{'id': 3457624, 'status': 'already in db'}
{'id': 3370779, 'status': 'already in db'}
{'id': 3205104, 'status': 'already in db'}
{'id': 3284271, 'status': 'already in db'}
{'id': 3292889, 'status': 'already in db'}
{'id': 3416674, 'status': 'already in db'}
{'id': 3257408, 'status': 'already in db'}
{'id': 3222300, 'status': 'already in db'}
{'id': 3382253, 'status': 'already in db'}
{'id': 3206312, 'status': 'already in db'}


### Запрос для тестирования панели 'Total Exception' дашборда 'FastAPI Service' в Grafana.
По этому запросу сервис ничего не делает. Только бросает исключение

In [43]:
url_raise_exception = 'http://127.0.0.1:8000/raise_exception'
for i in range(50):
    resp_raise_exception = requests.get(url_raise_exception)
resp_raise_exception.status_code

500

### Grafana

Посмотрим на дашборд 'FastAPI Service'

Grafana http://localhost:3000/

<center>
<img src="grafana_dashboard.JPG"> 
</center>